In [26]:
import pandas as pd
import socket
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle



In [27]:
model = LinearRegression()
test = pd.read_csv('/home/datascience/Desktop/Prac_438/data/test(1).csv')


In [28]:

HOST = '127.0.0.1'
PORT = 55872

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)

print("Server is listening...")

conn, addr = server_socket.accept()
print(f"Connected by: {addr}")

Server is listening...
Connected by: ('127.0.0.1', 38898)


In [29]:
##





df_list = []  # Store received chunks

try:
    while True:
        size_prefix = conn.recv(4)  # Read the first 4 bytes (data size)
        if not size_prefix:
            break
        
        data_size = int.from_bytes(size_prefix, 'big')  # Convert to integer
        data = b""
        while len(data) < data_size:
            chunk = conn.recv(data_size - len(data))  # Read full data
            if not chunk:
                break
            data += chunk

        # Deserialize and append to DataFrame list
        received_chunk = pickle.loads(data)
        df_list.append(pd.DataFrame(received_chunk))

        # Combine all chunks
        df = pd.concat(df_list, ignore_index=True)
        df.columns = ['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
        print("Received DataFrame:\n", df.tail())
        df.dropna()
        #df.fillna(0,inplace=True)
        
        features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
        target = 'fare_amount'
        X = df[features]
        y = df[target]
    
        X = X.dropna()
        y = y.loc[X.index] 
        X_test  = test[features]
        X_test = X_test.dropna()

    
        model.fit(X, y)


        y_pred = model.predict(X_test)

        #start=0
        #batch_size= len(y)
        #while start < len(data):
        #    end = min(start + batch_size, len(data))
        
        m = min(len(y),len(y_pred))
        mse = mean_squared_error(y[:m], y_pred[:m])
        print(f"Mean Squared Error: {mse}")
        
    
    
        print(f"Coefficients: {model.coef_}")
        print(f"Intercept: {model.intercept_}")
    
    # Here we can again send the final model weights and intersept back to the client
    #---TO BE DONE---
    

except KeyboardInterrupt:
    print("\nServer Stopped by user.")

finally:
    intercept = model.intercept_
    coef = model.coef_
    model = {
    "intercept":intercept,
    'coef':coef
    }
    model_info = pickle.dumps(model)
    conn.sendall(model_info)
    conn.close()
    server_socket.close()
    print("Connection Closed.")


Received DataFrame:
                                key fare_amount          pickup_datetime  \
995    2011-07-30 07:33:13.0000001        16.9  2011-07-30 07:33:13 UTC   
996  2011-09-24 23:20:00.000000112        13.7  2011-09-24 23:20:00 UTC   
997   2011-12-22 11:07:00.00000037        29.3  2011-12-22 11:07:00 UTC   
998   2014-11-03 12:40:00.00000014         6.5  2014-11-03 12:40:00 UTC   
999    2012-05-18 18:41:30.0000006        10.9  2012-05-18 18:41:30 UTC   

    pickup_longitude pickup_latitude dropoff_longitude dropoff_latitude  \
995       -73.884801       40.755707        -73.980472        40.765556   
996       -73.953603       40.779203        -73.995763        40.726701   
997        -73.94238       40.837712        -73.864372        40.769985   
998       -73.961151       40.774578        -73.972251         40.78564   
999       -73.988455       40.758432        -73.983816        40.730147   

    passenger_count  
995               1  
996               3  
997        

In [ ]:
model.predict(X_test)

array([11.27350495, 11.27421357, 11.27371142, ..., 11.80755948,
       11.81180851, 11.81038858], shape=(9914,))